<a href="https://colab.research.google.com/github/SumairaAmbreen/Contrroller/blob/main/fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the train and test data into pandas DataFrames
train_df = pd.read_csv(train_file_path, sep="\t", header=None, names=["label", "message"])
test_df = pd.read_csv(test_file_path, sep="\t", header=None, names=["label", "message"])

# Data Preprocessing

# Convert labels to binary: "ham" -> 0, "spam" -> 1
train_df['label'] = train_df['label'].map({"ham": 0, "spam": 1})
test_df['label'] = test_df['label'].map({"ham": 0, "spam": 1})

# Split the dataset into features (messages) and labels
X_train_texts = train_df['message'].values
y_train = train_df['label'].values
X_test_texts = test_df['message'].values
y_test = test_df['label'].values

# Tokenization and Padding

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train_texts)

# Convert text messages to sequences of integers
X_train = tokenizer.texts_to_sequences(X_train_texts)
X_test = tokenizer.texts_to_sequences(X_test_texts)

# Pad sequences to ensure consistent input length
# Pad sequences to ensure consistent input length
MAX_LENGTH = 100  # Set a fixed maximum length for padding
X_train = pad_sequences(X_train, padding='post', maxlen=MAX_LENGTH)
X_test = pad_sequences(X_test, padding='post', maxlen=MAX_LENGTH)


# Check for NaN values in the data
print("Training Data NaNs:", np.any(np.isnan(X_train)))
print("Testing Data NaNs:", np.any(np.isnan(X_test)))

# Replace NaNs with zeros if found
X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)


In [ ]:
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
# Build the model
def build_model():
    model = keras.Sequential([
        layers.Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=16, input_length=MAX_LENGTH),
        layers.GlobalAveragePooling1D(),
        layers.Dense(16, activation="relu"),
        layers.Dense(1, activation="sigmoid")  # Sigmoid for binary classification (ham or spam)
    ])

    # Compile the model with a lower learning rate to avoid NaN loss
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),  # Increase learning rate
                  loss="binary_crossentropy", metrics=["accuracy"])

    return model

# Initialize and train the model
model = build_model()
# Define EarlyStopping callback to prevent overfitting
early_stopping = EarlyStopping(monitor='loss', patience=3)

# Train the model
history = model.fit(X_train, y_train, epochs=20, batch_size=32,
                    validation_data=(X_test, y_test),
                    verbose=2,
                    callbacks=[early_stopping])  # Early stopping to prevent overfitting

# Plot the training and validation loss and accuracy
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.show()

plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.show()


In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
# Prediction function

def predict_message(pred_text):
    # Tokenize and pad the input message
    seq = tokenizer.texts_to_sequences([pred_text])
    padded_seq = pad_sequences(seq, padding='post', maxlen=X_train.shape[1])

    # Make a prediction
    prediction = model.predict(padded_seq)

    # Return the prediction probability and the corresponding label
    return [float(prediction[0][0]), "spam" if prediction[0][0] > 0.5 else "ham"]

# Example usage
pred_text = "how are you doing today?"
prediction = predict_message(pred_text)
print("Prediction for '{}':".format(pred_text), prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


In [ ]:
{
  "cells": [...],
  "metadata": {
    "kernelspec": {
      "display_name": "Python 3",
      "language": "python",
      "name": "python3"
    },
    "name": "fcc_sms_text_classification.ipynb"
  },
  "nbformat": 4,
  "nbformat_minor": 5
}
